# 自製智能中文選字系統  (1)

## 資料前處理

確認版本為 python3

In [10]:
import re

In [11]:
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    segments = re.findall(r'[\u4E00-\u9FFF]+', line)

    return segments

In [12]:
prepocess_line('“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')  
# 應該為：['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

In [13]:
with open('wiki_zh_small.txt', encoding='utf-8') as f:
    segments = sum([prepocess_line(line) for line in f.readlines()], [])

## Ngram

一開始要先計算字詞出現的次數

In [14]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        # 分別代表計算0、1、...個字的出現次數
        self.counters = [Counter() for _ in range(n + 1)]

    def fit(self, segments):
        # 因為 self.counters 分別代表計算0、1、...個字的出現次數
        # 請在此實作利用 segments 以及函式 _skip 來統計次數
        for i in range(self.n):
            for segment in segments:
                self.counters[i + 1].update(self._skip(segment, i + 1))
        self.counters[0] = Counter({'': sum([len(segment) for segment in segments])})

    def __getitem__(self, k):
        return self.counters[k]

    def _skip(self, segment, n):
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i + shift + 1]

In [15]:
counters = Counters(n=3)
counters.fit(segments)

In [16]:
counters[0]
# 應該為： Counter({'': 371373})

Counter({'': 371373})

In [17]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]
        self.minor_counter = counters[n - 1]

    def predict_proba(self, prefix='', top_k=5):
        assert len(prefix) >= self.n - 1
        # 使用 Ngram 的公式計算出下一個字出現的機率
        # 輸出為機率與字的tuple列表，詳見下方輸出範例
        prev = prefix[-(self.n - 1):] if self.n > 1 else ''
        prev_cnt = self.minor_counter[prev]
        probs = []
        for key, val in self.major_counter.items():
            if key.startswith(prev):
                probs.append((val / prev_cnt, key[-1]))
        sorted_probs = sorted(probs, reverse=True)

        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}


In [18]:
unigram = Ngram(1, counters)

In [19]:
unigram.predict_proba('我思')
# 應該為：[(0.035732269174118744, '的'),
#         (0.012927703414087723, '國'),
#         (0.010620050461395955, '中'),
#         (0.009984570768472667, '在'),
#         (0.009852627950874188, '一')]

[(0.035732269174118744, '的'),
 (0.012927703414087723, '國'),
 (0.010620050461395955, '中'),
 (0.009984570768472667, '在'),
 (0.009852627950874188, '一')]

In [20]:
bigram = Ngram(2, counters)
trigram = Ngram(3, counters)

## 使用Ngram來建立第一版選字系統

In [21]:
class ChineseWordRecommenderV1:
    def __init__(self, unigram, bigram, trigram):
        self.unigram = unigram
        self.bigram = bigram
        self.trigram = trigram
    
    def predict_proba(self, prefix='', top_k=5):
        # 使用Ngram來建立選字系統
        if len(prefix) == 0:
            return self.unigram.predict_proba(prefix, top_k=top_k)
        elif len(prefix) == 1:
            return self.bigram.predict_proba(prefix, top_k=top_k)
        else:
            return self.trigram.predict_proba(prefix, top_k=top_k)

In [22]:
model = ChineseWordRecommenderV1(unigram, bigram, trigram)

In [23]:
probs = model.predict_proba('我思', top_k=10)
probs

[(0.75, '故'), (0.25, '維')]

## Demo

In [24]:
!pip install -U pip
!pip install -q ipywidgets

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 存取被拒。: 'C:\\Users\\user\\AppData\\Local\\Temp\\pip-uninstall-6a2iz41u\\pip.exe'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1


In [25]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')